In [1]:
import requests
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
class weatherForecast:
    def __init__(self, hours = 48):
        self.ready = False
        self.PARAMS = {
            'lang':'en',
            'hours':hours 
        }
        
        self.temp = pd.DataFrame()
        self.weather_forecast = pd.DataFrame()
        
        self.data = []
        
        self.input_data = pd.DataFrame(columns = ['Type', 'Location', 'Status', 'City', 'Name'])
        
        seasons = pd.read_csv('seasons.csv')
        seasons['StartDate']= pd.to_datetime(seasons['StartDate']).dt.date
        seasons['EndDate']= pd.to_datetime(seasons['EndDate']).dt.date
        self.seasons = np.array(seasons.to_records(index = False))

        weather_codes = pd.read_csv("weather_codes.csv")
        weather_codes.drop(labels='Description ',axis = 1, inplace = True)
        self.weather_codes = dict(zip(weather_codes['Code '], weather_codes.Weather))
        
        holidays = pd.read_csv('Holidays 2019.csv')
        self.holidays = pd.to_datetime(holidays['Date']).dt.date.values
        
    def filterP(self, place, date, hr):
        if not place in list(self.input_data['Name']):
            print("Not in input data yet, please add first")
            print(list(self.input_data['Place']))
            return
        
        date = dt.datetime.strptime(date, '%Y-%m-%d').date()
        dattim = dt.datetime.combine(date, dt.time(hr, 30))
        
        ans = self.weather_forecast[(self.weather_forecast['place'] == place) & (self.weather_forecast['timestamp_local'] == dattim)]
        
        if(ans.shape[0] == 0):
            print("No results found")
            return -1
        if ans.shape[0] > 1:
            print("Too many results(?)")
            print(ans)
            return -2
        
        return ans
    
        
        
    def inputData(self, typeloc, loc, nameofPlace='NA',hours = 48):
        if not typeloc in ['latlon']:
            return 
        
        if(typeloc == 'latlon'):
            self.PARAMS = {
                'lat':loc[0],
                'lon':loc[1],
                
                'lang':'en',
                'hours':hours 
            }
            
            self.ready = True
            x = self.callAPI()
            self.ready = False

            if not x.status_code == 200:
                print("Invalid response",x.status_code)
                return
            self.data = x.json()

            from pandas.io.json import json_normalize
            self.temp =  json_normalize(self.data['data'])
            
            loc = [typeloc,loc,1, self.data['city_name'], nameofPlace]
            self.input_data = self.input_data.append(pd.DataFrame([loc], columns = ['Type', 'Location', 'Status', 'City', 'Name']))
            self.modifyDf(nameofPlace)
            self.addDf()
            
            
    def addDf(self):
        if self.weather_forecast.empty:
            self.weather_forecast = self.temp.copy()
        else:
            self.weather_forecast = self.weather_forecast.append(self.temp, ignore_index = True)
            
        self.temp = pd.DataFrame()
        
        
    def getSeasonc(self, dateGive):
        Y = 2000
        if isinstance(dateGive, dt.datetime):
            dateGive = dateGive.date()
        else:
            raise TypeError("wrong Type Given")
        dateGive = dateGive.replace(year=Y)
        return next((season,code) for start, end,season, fancy,code in self.seasons
                    if start <= dateGive <= end)


    def checkHolc(self, dateGive):
        if isinstance(dateGive, dt.datetime):
            dateGive = dateGive.date()

        if not isinstance(dateGive, dt.date):
            raise TypeError("wrong Type Given")

        if dateGive in self.holidays:
            return 1
        return 0

    
    def callAPI(self):
        if not self.ready:
            print("Not Reayd")
            return
        
        r = requests.get("https://weatherbit-v1-mashape.p.rapidapi.com/forecast/hourly",
                         headers={
                             "X-RapidAPI-Host": "weatherbit-v1-mashape.p.rapidapi.com",
                             "X-RapidAPI-Key": "9760fa76cemsh3b3bc0cfd34ff97p14b793jsnacc85f71e316"
                         },
                         params = self.PARAMS
        )
        return r
        

    def modifyDf(self, place):
        self.temp['datetime'] = pd.to_datetime(self.temp['datetime'], format = "%Y-%m-%d:%H")
        self.temp['timestamp_local'] = pd.to_datetime(self.temp['timestamp_local'])
        self.temp['city'] = self.data['city_name']
        self.temp['lon'] = self.data['lon']
        self.temp['lat'] = self.data['lat']
        self.temp['place'] = place

        self.temp['weather_code'] = self.temp['weather.code'].apply(lambda x : int(self.weather_codes[x]))

    
        self.temp['season'], self.temp['season_code'] = zip(*self.temp['timestamp_local'].apply(self.getSeasonc))
        self.temp['dayofweek'] = self.temp['timestamp_local'].dt.dayofweek
        self.temp['workingday'] = self.temp['dayofweek']//5

        self.temp['holiday'] = self.temp['timestamp_local'].dt.date.apply(self.checkHolc)
        
        self.temp['year'] = self.temp['timestamp_local'].dt.year
        self.temp['month'] = self.temp['timestamp_local'].dt.month
        self.temp['day'] = self.temp['timestamp_local'].dt.hour
        self.temp['hour'] = self.temp['timestamp_local'].dt.hour


    def displayReq(self):
        print(self.temp[['place','lat', 'lon', 'city','app_temp', 'timestamp_local', 'temp', 'rh', 'wind_spd', 'weather_code','season', 'season_code','dayofweek', 'workingday','holiday', 'year', 'month', 'day', 'hour']])



In [3]:
l = weatherForecast()

In [4]:
l.inputData('latlon', [12.9304,77.6784], 'Bellandur', 120)

In [5]:
l.inputData('latlon', [12.9063,77.5857], 'JP Nagar')

In [6]:
l.input_data

,Type,Location,Status,City,Name
0,latlon,"[12.9304, 77.6784]",1,Bengaluru,Bellandur
0,latlon,"[12.9063, 77.5857]",1,Bengaluru,JP Nagar


In [7]:
l.filterP('Bellandur', '2019-07-26', 00)

,app_temp,clouds,clouds_hi,clouds_low,clouds_mid,datetime,dewpt,dhi,dni,ghi,...,weather_code,season,season_code,dayofweek,workingday,holiday,year,month,day,hour
83,22.3,100,100,0,36,2019-07-25 19:00:00,18.3,0.0,0.0,0.0,...,1,Monsoon,3,4,0,0,2019,7,0,0


In [8]:
l.weather_forecast[['place','lat', 'lon', 'city','app_temp', 'timestamp_local', 'temp', 'rh', 'wind_spd', 'weather_code','season', 'season_code','dayofweek', 'workingday','holiday', 'year', 'month', 'day', 'hour']]

,place,lat,lon,city,app_temp,timestamp_local,temp,rh,wind_spd,weather_code,season,season_code,dayofweek,workingday,holiday,year,month,day,hour
0,Bellandur,12.93,77.68,Bengaluru,24.3,2019-07-22 13:30:00,23.9,75,5.83772,4,Monsoon,3,0,0,0,2019,7,13,13
1,Bellandur,12.93,77.68,Bengaluru,24.0,2019-07-22 14:30:00,23.6,77,7.14698,4,Monsoon,3,0,0,0,2019,7,14,14
2,Bellandur,12.93,77.68,Bengaluru,22.7,2019-07-22 15:30:00,22.3,84,6.72859,4,Monsoon,3,0,0,0,2019,7,15,15
3,Bellandur,12.93,77.68,Bengaluru,22.0,2019-07-22 16:30:00,21.4,89,6.43049,4,Monsoon,3,0,0,0,2019,7,16,16
4,Bellandur,12.93,77.68,Bengaluru,21.8,2019-07-22 17:30:00,21.3,90,6.41371,4,Monsoon,3,0,0,0,2019,7,17,17
5,Bellandur,12.93,77.68,Bengaluru,21.6,2019-07-22 18:30:00,21.1,92,5.73771,4,Monsoon,3,0,0,0,2019,7,18,18
6,Bellandur,12.93,77.68,Bengaluru,21.4,2019-07-22 19:30:00,20.9,93,5.19701,4,Monsoon,3,0,0,0,2019,7,19,19
7,Bellandur,12.93,77.68,Bengaluru,21.4,2019-07-22 20:30:00,20.9,94,5.40485,4,Monsoon,3,0,0,0,2019,7,20,20
8,Bellandur,12.93,77.68,Bengaluru,21.1,2019-07-22 21:30:00,20.6,93,5.69551,4,Monsoon,3,0,0,0,2019,7,21,21
9,Bellandur,12.93,77.68,Bengaluru,21.1,2019-07-22 22:30:00,20.6,93,7.61460,4,Monsoon,3,0,0,0,2019,7,22,22
